# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
# import libraries
import DataScienceHelperLibrary as dsh

import numpy as np
import pandas as pd
import re

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion


from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
origDb = 'DisasterResponse.db'
origMessages = 'Message'

engine = create_engine('sqlite:///{}'.format(origDb))

df = pd.read_sql_table(origMessages, engine)

In [3]:
dsh.AnalyzeDataFrame(df)

-------------------- Dataframe analysis started --------------------
Shape:  (26216, 41)
Number of duplicate rows:  0
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 40, 97.56%
index, type: int64                     0.0
id, type: int64                        0.0
message, type: object                  0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
child_alone, type: int64               0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64                   0.0
clothing, type: int64              

In [4]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [5]:
df[df.genre == 'direct']

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
8,8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1


In [6]:
df

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
8,8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1


We don't need the original text because all messages are given.

In [7]:
df = dsh.RemoveColumnsByWildcard(df, 'original')

-------------------- Start finding and removing columns matchting to wildcards: ['original'] --------------------
Columns found to remove:  ['original']
-------------------- Finished removing columns matchting to wildcards --------------------


In [48]:
dfsub = df[(df.message.str.startswith('NOTE'))]['message']
#(df.related > 1) &
for val in list(dfsub):
    print(val)

NOTES: Personal. Not an emergency.
NOTES: aclet dry We are counting on you.
NOTES: talking about gas prices
NOTES: Historic earthquake in Haiti.
NOTES: They are requesting that somebody to pass by Butte Boyer #3 They don't have water food and Medicines
NOTES: Regular gossip or message sharing. Not an emergency.
NOTES: personal request. Not an emergency.
NOTES: regular gossip. Not an emergency.
NOTES: Gossip. Not an emergency.
NOTES: Personal comment. Not an emergency.
NOTES: Personal opinion. Not an emergency.
NOTES: Inquiry. No action needed at this time
NOTES: looking for work - construction
NOTES: Concerns about remainder of school year
NOTES: Gossip. Personal. Not an emergency.
NOTES: Discard. Not an emergency.
NOTES: Personal concerns and offer of help to ditribute goods.
NOTES: An opinion and request to the Gvt
NOTES: Comment and political opinion. No emergency.
NOTES: Statement. No emergency.
NOTES: The stench of death is overwhelming the citizens
NOTES: I already translated thi

In [9]:
'''
for i in range(dfsub.shape[0]):
    dsh.PrintLine('Row: ' + str(i))
    for j in range(dfsub.shape[1]):
        if dfsub.columns[j] in ['id', 'index']:
            continue
        if dfsub.iloc[i, j] == 0:
            continue
        print(dfsub.columns[j], ' - ', dfsub.iloc[i, j])
'''

"\nfor i in range(dfsub.shape[0]):\n    dsh.PrintLine('Row: ' + str(i))\n    for j in range(dfsub.shape[1]):\n        if dfsub.columns[j] in ['id', 'index']:\n            continue\n        if dfsub.iloc[i, j] == 0:\n            continue\n        print(dfsub.columns[j], ' - ', dfsub.iloc[i, j])\n"

In [10]:
X = dsh.SelectColumnsByWildcard(df, ['message'])

Y = dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message', 'original', 'genre'])

-------------------- Start finding and keeping columns matchting to wildcards: ['message'] --------------------
-------------------- Finished keeping columns matchting to wildcards: 1 --------------------
-------------------- Start finding and removing columns matchting to wildcards: ['index', 'id', 'message', 'original', 'genre'] --------------------
Columns found to remove:  ['index', 'id', 'message', 'genre']
-------------------- Finished removing columns matchting to wildcards --------------------


### 2. Write a tokenization function to process your text data

In [49]:
# 
regUrl = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
regChars = r'[^a-zA-Z0-9]'


def Tokenize(text):
    
    text = text.lower()
    
    #text = re.sub(regUrl, " ", text)
    text = re.sub(regChars, " ", text)
    
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    
    cleanTokens = []
    for tok in tokens:
        cleanTok = lemmatizer.lemmatize(tok).strip()
        #cleanTok = [PorterStemmer().stem(tok) for tok in cleanTok]
        
        if cleanTok in stopwords.words('english'):
            continue
        
        cleanTokens.append(cleanTok)
    
    return cleanTokens
        
        

In [50]:
Tokenize('0H4Z(Z x(@zf0 ?!?@x PrxP0c6 ')

['0h4z', 'z', 'x', 'zf0', 'x', 'prxp0c6']

In [41]:
vals = list(df[df.genre == 'social'].message[:10])
for val in vals:
    dsh.PrintLine()
    print(val)
    print(Tokenize(val))
    dsh.PrintLine()

--------------------  --------------------
My thoughts and prayers go out to all the lives that were affected today by the earthquake in Haiti. I will keep you all in my prayers!
['thought', 'prayer', 'go', 'life', 'affected', 'today', 'earthquake', 'haiti', 'keep', 'prayer']
--------------------  --------------------
--------------------  --------------------
I m sorry for the poor people in Haiti tonight. That being said I d love to experience an earthquake just once. In a big field of course.
['sorry', 'poor', 'people', 'haiti', 'tonight', 'said', 'love', 'experience', 'earthquake', 'big', 'field', 'course']
--------------------  --------------------
--------------------  --------------------
RT selenagomez UNICEF has just announced an emergency alert for the people of Haiti who were hit by a 7.0 earthquake and a tsunami.. ..
['rt', 'selenagomez', 'unicef', 'ha', 'announced', 'emergency', 'alert', 'people', 'haiti', 'hit', '7', '0', 'earthquake', 'tsunami']
--------------------  ---

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [55]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = Tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf', MultiOutputClassifier(estimator = #TODO ))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [56]:
XTrain, XTest, yTrain, yTest = dsh.SplitDataTrainTest(X, Y)

In [57]:
print(XTrain.shape)
print(XTest.shape)
print(yTrain.shape)
print(yTest.shape)

(18351, 1)
(7865, 1)
(18351, 36)
(7865, 36)


In [58]:
pipeline.fit(X = X, y = Y)

ValueError: The base estimator should implement a fit method

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.